In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# Original recursive phasor calculation function
def phasor_calculation_recursive(Nw, amp, phase, f, N, f0):
    Nt = N * Nw  # Total number of samples
    fs = f0 * N  # Sampling frequency
    dt = 1 / fs
    t = dt * np.arange(Nt)

    fs1 = N * f
    dt1 = 1 / fs1
    t1 = dt1 * np.arange(Nt)

    x = amp * np.cos(2 * np.pi * f * t + phase)
    x1 = amp * np.cos(2 * np.pi * f * t1 + phase)

    X = np.zeros(Nw, dtype=complex)  # Initialize phasor array

    th = 2 * np.pi / N
    df = f - f0
    w0 = 2 * np.pi * f0
    dw = 2 * np.pi * df
    w = w0 + dw

    if dw == 0:
        P = 1
        Q = 0
    else:
        P = (np.sin(N * (w - w0) * dt / 2) / (N * np.sin((w - w0) * dt / 2))) * np.exp(1j * (N - 1) * (w - w0) * dt / 2)
        Q = (np.sin(N * (w + w0) * dt / 2) / (N * np.sin((w + w0) * dt / 2))) * np.exp(-1j * (N - 1) * (w + w0) * dt / 2)

    # Calculate phasor for the first window
    for n in range(N):
        X[0] += (np.sqrt(2) / N) * x1[n] * np.exp(-1j * n * th)

    # Recursively calculate phasors for subsequent windows
    for k in range(1, Nw):
        X[k] = X[k - 1] + (np.sqrt(2) / N) * (x1[N + k - 1] - x1[k - 1]) * np.exp(-1j * k * th)

    Xb = np.zeros(Nw, dtype=complex)
    for k in range(Nw):
        Xb[k] = P * X[k] * np.exp(1j * k * (w - w0) * dt) + Q * np.conj(X[k]) * np.exp(-1j * k * (w + w0) * dt)

    return Xb

# Web version of the phasor analysis tool
class WebPhasorApp:
    def __init__(self):
        # Initialize variables
        self.enable_three_point_avg = False
        self.recursive_phasors = []
        self.sample_index = 0
        self.total_phase_recursive = 0
        self.non_recursive_phasors = []
        self.signal_data = []
        self.sample_count = 0
        self.window_data = []
        self.current_window_index = 0
        self.is_first_calculation = True
        
        # Create input widgets
        self.amplitude_widget = widgets.FloatText(value=10, description='Amplitude:')
        self.frequency_widget = widgets.FloatText(value=51, description='Frequency (Hz):')
        self.nominal_frequency_widget = widgets.FloatText(value=50, description='Nominal Freq (Hz):')
        self.samples_widget = widgets.IntText(value=20, description='Samples/Cycle:')
        self.phase_widget = widgets.FloatText(value=0, description='Phase (°):')
        
        # Create buttons
        self.calculate_button = widgets.Button(description='Calculate')
        self.clear_button = widgets.Button(description='Clear')
        
        # Three-point average checkbox
        self.three_point_avg_checkbox = widgets.Checkbox(
            value=False,
            description='Enable 3-Point Avg',
            disabled=False
        )
        
        # Phasor value display
        self.status_output = widgets.HTML(value="Ready - Enter parameters and click Calculate")
        self.recursive_re_output = widgets.HTML(value="Recursive Phasor Re: 0.0")
        self.recursive_im_output = widgets.HTML(value="Recursive Phasor Im: 0.0")
        self.non_recursive_re_output = widgets.HTML(value="Non-Recursive Phasor Re: 0.0")
        self.non_recursive_im_output = widgets.HTML(value="Non-Recursive Phasor Im: 0.0")
        
        # Create output area
        self.output = widgets.Output()
        
        # Configure button click handlers
        self.calculate_button.on_click(self.on_calculate_button_clicked)
        self.clear_button.on_click(self.on_clear_button_clicked)
        self.three_point_avg_checkbox.observe(self.on_checkbox_change, names='value')
        
        # Create layout
        input_box = widgets.VBox([
            self.amplitude_widget,
            self.frequency_widget,
            self.nominal_frequency_widget, 
            self.samples_widget,
            self.phase_widget,
            widgets.HBox([self.calculate_button, self.clear_button]),
            self.three_point_avg_checkbox,
            widgets.VBox([
                self.recursive_re_output,
                self.recursive_im_output,
                self.non_recursive_re_output,
                self.non_recursive_im_output,
                self.status_output
            ])
        ])
        
        # Build UI with layout
        self.ui = widgets.HBox([input_box, self.output])
    
    def display(self):
        """Display the application UI"""
        display(self.ui)
    
    def on_checkbox_change(self, change):
        """Handle three-point average checkbox changes"""
        self.enable_three_point_avg = change['new']
        status = "Enabled" if self.enable_three_point_avg else "Disabled"
        self.status_output.value = f"Three-Point Average {status}"
    
    def on_clear_button_clicked(self, b):
        """Clear button callback"""
        with self.output:
            clear_output(wait=True)
        
        # Reset all data
        self.recursive_phasors = []
        self.total_phase_recursive = 0
        self.sample_index = 0
        
        self.non_recursive_phasors = []
        self.signal_data = []
        self.sample_count = 0
        
        self.window_data = []
        self.current_window_index = 0
        self.is_first_calculation = True
        
        # Reset display
        self.recursive_re_output.value = "Recursive Phasor Re: 0.0"
        self.recursive_im_output.value = "Recursive Phasor Im: 0.0"
        self.non_recursive_re_output.value = "Non-Recursive Phasor Re: 0.0"
        self.non_recursive_im_output.value = "Non-Recursive Phasor Im: 0.0"
        self.status_output.value = "Cleared - Ready for new calculation"
        
        # Reset three-point average
        self.three_point_avg_checkbox.value = False
        self.enable_three_point_avg = False
    
    def prefill_window(self):
        """Prefill the signal window, generating N valid signal samples"""
        amp = self.amplitude_widget.value
        f0 = self.nominal_frequency_widget.value
        N = self.samples_widget.value
        phase = self.phase_widget.value
        
        # Clear existing data
        self.signal_data = []
        self.sample_count = 0
        
        # Sampling frequency and time interval
        fs = f0 * N
        dt = 1 / fs
        
        # Pre-generate N signal samples
        for i in range(N):
            sample_time = self.sample_count * dt
            self.sample_count += 1
            sample = amp * np.cos(2 * np.pi * f0 * sample_time + np.deg2rad(phase))
            self.signal_data.append(sample)
    
    def phasor_calculation_non_recursive(self, amp, phase, f0, N):
        """Calculate non-recursive phasor based on sliding window"""
        fs = f0 * N  # Sampling frequency
        dt = 1 / fs   # Sampling time interval
        
        # Use sample counter to generate continuous time points
        new_sample_time = self.sample_count * dt
        self.sample_count += 1
        
        # Generate new sample
        new_sample = amp * np.cos(2 * np.pi * f0 * new_sample_time + np.deg2rad(phase))  

        # Sliding window management
        if len(self.signal_data) >= N:
            self.signal_data.pop(0)  # Remove oldest sample
        self.signal_data.append(new_sample)  # Add new sample

        # Zero-padding to match N length
        signal_array = np.array(self.signal_data)
        if len(signal_array) < N:
            signal_array = np.pad(signal_array, (N - len(signal_array), 0), mode='constant')

        # Calculate phasor
        theta = 2 * np.pi / N  # Angular increment
        X = np.sum((np.sqrt(2) / N) * signal_array * np.exp(-1j * np.arange(N) * theta))

        return X
    
    def calculate_recursive_phasor(self):
        """Calculate recursive phasor"""
        amp = self.amplitude_widget.value
        f = self.frequency_widget.value
        f0 = self.nominal_frequency_widget.value
        N = self.samples_widget.value
        phase = np.deg2rad(self.phase_widget.value)
        Nw = 2  # Fixed at 2
        
        # Calculate phase increment (per sample)
        sample_time = 1 / (f0 * N)  # Time per sample
        df = f - f0
        theoretical_increment = 2 * np.pi * df * sample_time  # Theoretical phase increment per sample
        
        # Accumulate phase
        if self.sample_index == 0:
            # First calculation, initialize phase
            Xb = phasor_calculation_recursive(Nw, amp, phase, f, N, f0)
            current_phasor = Xb[0]  # Use first phasor
            self.total_phase_recursive = np.angle(current_phasor)
        else:
            # Subsequent calculations, accumulate phase
            self.total_phase_recursive += theoretical_increment
        
        # Create phasor with accumulated phase
        new_phasor = amp * np.exp(1j * self.total_phase_recursive)
        
        # Store new phasor and update index
        self.recursive_phasors.append(new_phasor)
        self.sample_index += 1  # Slide one sample
        
        return new_phasor, df, theoretical_increment

    def calculate_non_recursive_phasor(self):
        """Calculate non-recursive phasor"""
        amp = self.amplitude_widget.value
        f0 = self.nominal_frequency_widget.value
        N = self.samples_widget.value
        phase = self.phase_widget.value
        
        # Check if window is filled, prefill if not
        if len(self.signal_data) < N:
            self.prefill_window()
        
        # Calculate new phasor
        new_phasor = self.phasor_calculation_non_recursive(amp, phase, f0, N)
        
        # Store calculated phasor
        self.non_recursive_phasors.append(new_phasor)
        
        return new_phasor
    
    def generate_data_if_needed(self):
        """Generate data as needed to support infinite sliding"""
        # Get input parameters
        V_m = self.amplitude_widget.value
        f = self.frequency_widget.value
        f_0 = self.nominal_frequency_widget.value
        N = self.samples_widget.value
        phi = self.phase_widget.value * np.pi / 180

        # If first calculation, initialize parameters and generate initial dataset
        if self.is_first_calculation:
            # Calculate sampling interval
            Delta_t = 1 / (N * f_0)
            
            # Initially generate 50 cycles of data (adjust as needed)
            n_periods = 50
            total_samples = n_periods * N
            t = np.arange(0, total_samples * Delta_t, Delta_t)
            
            # Generate signal data
            self.window_data = V_m * np.cos(2 * np.pi * f * t + phi)
            
            # Store parameters for subsequent calculations
            self.params = {
                'V_m': V_m,
                'f': f,
                'f_0': f_0,
                'N': N,
                'phi': phi,
                'Delta_t': Delta_t
            }
            
            self.is_first_calculation = False
            return True
        
        # Check if we need to generate more data
        if self.current_window_index + self.params['N'] * 2 > len(self.window_data):
            # Need to generate more data, add 20 cycles each time
            n_additional_periods = 20
            additional_samples = n_additional_periods * self.params['N']
            
            # Calculate start time for new data
            start_time = len(self.window_data) * self.params['Delta_t']
            end_time = start_time + additional_samples * self.params['Delta_t']
            t_new = np.arange(start_time, end_time, self.params['Delta_t'])
            
            # Generate new data points
            new_data = self.params['V_m'] * np.cos(2 * np.pi * self.params['f'] * t_new + self.params['phi'])
            
            # Append new data to existing data
            self.window_data = np.append(self.window_data, new_data)
            
            return True
        
        return False
        
    def on_calculate_button_clicked(self, b):
        """Calculate button callback"""
        try:
            with self.output:
                clear_output(wait=True)
                
                # Create figure
                fig = plt.figure(figsize=(12, 12))
                
                # Calculate recursive phasor
                recursive_phasor, df, theoretical_increment = self.calculate_recursive_phasor()
                
                # Calculate non-recursive phasor
                non_recursive_phasor = self.calculate_non_recursive_phasor()
                
                # Check if window data needs to be generated
                self.generate_data_if_needed()
                
                # Get parameters
                N = self.samples_widget.value
                f_0 = self.nominal_frequency_widget.value
                Delta_t = 1 / (N * f_0)
                f = self.frequency_widget.value
                
                # Get current window data
                current_window = self.window_data[self.current_window_index:self.current_window_index + N]
                
                # Calculate time axis for current window
                t_window = np.arange(self.current_window_index, self.current_window_index + N) * Delta_t
                
                # Calculate angular frequencies
                omega_0 = 2 * np.pi * f_0  
                omega = 2 * np.pi * f  
                
                # Calculate sampling interval
                duration = 2 / f  
                t = np.linspace(t_window[0], t_window[0] + duration, int(N * 2))  
                sampled_t = t_window
                
                # Generate signal
                signal = self.amplitude_widget.value * np.cos(2 * np.pi * f * t + np.deg2rad(self.phase_widget.value))
                sampled_signal = current_window

                # FFT spectrum
                N_fft = len(sampled_signal)
                signal_fft = np.fft.fft(sampled_signal)
                signal_fft = np.fft.fftshift(signal_fft)  
                f_plot = np.linspace(-1/(2*Delta_t), 1/(2*Delta_t), N_fft)  

                # Calculate recursive phasors
                n_end = 200  
                # Use start time of current window
                start_time = self.current_window_index * Delta_t
                t_true = np.arange(start_time, start_time + n_end * Delta_t, Delta_t)
                x_n_true = np.sqrt(2) * self.amplitude_widget.value * np.cos(omega * t_true + np.deg2rad(self.phase_widget.value))

                def get_phasor(x_w, N):
                    X_N_v = x_w * np.exp(-1j * np.arange(N) * (2 * np.pi / N))
                    return (np.sqrt(2) / N) * np.sum(X_N_v)

                X_hat_0 = get_phasor(x_n_true[:N], N)
                X_hat_prev = X_hat_0
                X_hat_v = [X_hat_prev]

                j = N
                while j < n_end - N + 1:
                    r = j - N
                    X_hat_i = X_hat_prev + (np.sqrt(2) / N) * (x_n_true[N + r] - x_n_true[r]) * np.exp(-1j * r * (2 * np.pi / N))
                    X_hat_v.append(X_hat_i)
                    X_hat_prev = X_hat_i
                    j += 1

                # Convert to NumPy array
                X_hat_v = np.array(X_hat_v)
                
                # Store original X_hat_v for comparison
                X_hat_v_original = X_hat_v.copy()
                
                # Apply three-point average to X_hat_v if enabled
                if self.enable_three_point_avg and len(X_hat_v) > 1:
                    N = self.samples_widget.value
                    k60 = round(N / 6)  # 60 degree phase shift point (1/6 of cycle)
                    k120 = round(N / 3)  # 120 degree phase shift point (1/3 of cycle)
                    
                    # Create filtered version of X_hat_v
                    X_hat_v_filtered = np.copy(X_hat_v)
                    
                    # Only apply averaging when we have enough points
                    if len(X_hat_v) > k120 + 1:
                        # Apply three-point average to each element after first k120 points
                        for i in range(k120, len(X_hat_v)):
                            X_hat_v_filtered[i] = (X_hat_v[i] + X_hat_v[i-k60] + X_hat_v[i-k120]) / 3
                        
                        # Store filtered data
                        X_hat_v_filtered_for_display = X_hat_v_filtered.copy()
                        # Use filtered data for subsequent calculations
                        X_hat_v = X_hat_v_filtered
                else:
                    # If not enabled, they are the same
                    X_hat_v_filtered_for_display = X_hat_v
                
                # Create responsive grid specification
                gs = GridSpec(3, 2, figure=fig, height_ratios=[1.5, 1, 1])
                
                # Polar plots on top row
                ax_polar1 = fig.add_subplot(gs[0, 0], projection='polar')
                ax_polar2 = fig.add_subplot(gs[0, 1], projection='polar')
                
                # Time domain on middle left
                ax_time = fig.add_subplot(gs[1, 0])
                
                # FFT spectrum on middle right
                ax_fft = fig.add_subplot(gs[1, 1])
                
                # Recursive phasor magnitude on bottom left
                ax_mag = fig.add_subplot(gs[2, 0])
                
                # Recursive phasor phase on bottom right
                ax_phase = fig.add_subplot(gs[2, 1])
                
                # Set chart titles
                ax_polar1.set_title("Recursive Phasor", fontsize=12)
                ax_polar2.set_title("Non-Recursive Phasor", fontsize=12)
                
                # Plot recursive phasors - historical phasors in black
                for i, phasor in enumerate(self.recursive_phasors[:-1]):
                    magnitude = np.abs(phasor)
                    angle = np.angle(phasor)
                    # Use black, smaller dots and thin lines
                    ax_polar1.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                   linewidth=0.8, color='black', alpha=0.7)
                
                # Plot latest recursive phasor in red
                if self.recursive_phasors:
                    latest_recursive = self.recursive_phasors[-1]
                    magnitude = np.abs(latest_recursive)
                    angle = np.angle(latest_recursive)
                    # Use red, larger dots and thicker lines
                    ax_polar1.plot([0, angle], [0, magnitude], marker='.', markersize=4, linestyle='-', 
                                   linewidth=1.5, color='red')
                    
                    # Add label for latest recursive phasor
                    angle_deg = np.degrees(self.total_phase_recursive) % 360
                    ax_polar1.text(angle, magnitude * 1.1, f"{angle_deg:.1f}°", 
                                  horizontalalignment='center', verticalalignment='bottom', color='red', fontsize=8)
                
                # Plot non-recursive phasors - historical phasors in black
                for i, phasor in enumerate(self.non_recursive_phasors[:-1]):
                    magnitude = np.abs(phasor)
                    angle = np.angle(phasor)
                    # Use black, smaller dots and thin lines
                    ax_polar2.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                   linewidth=0.8, color='black', alpha=0.7)
                
                # Plot latest non-recursive phasor in red
                if self.non_recursive_phasors:
                    latest_non_recursive = self.non_recursive_phasors[-1]
                    magnitude = np.abs(latest_non_recursive)
                    angle = np.angle(latest_non_recursive)
                    # Use red, smaller dots and thin lines
                    ax_polar2.plot([0, angle], [0, magnitude], marker='.', markersize=3, linestyle='-', 
                                   linewidth=1.0, color='red')
                    
                    # Add label for latest non-recursive phasor
                    angle_deg = np.degrees(angle)
                    ax_polar2.text(angle, magnitude * 1.1, f"{angle_deg:.1f}°", 
                                  horizontalalignment='center', verticalalignment='bottom', color='red', fontsize=8)
                
                # Set chart ranges to ensure proper display
                max_recursive_magnitude = max([np.abs(p) for p in self.recursive_phasors]) if self.recursive_phasors else 1
                max_non_recursive_magnitude = max([np.abs(p) for p in self.non_recursive_phasors]) if self.non_recursive_phasors else 1
                ax_polar1.set_rmax(max_recursive_magnitude * 1.2)
                ax_polar2.set_rmax(max_non_recursive_magnitude * 1.2)
                
                # Adjust tick label size for better visibility on smaller screens
                ax_polar1.tick_params(labelsize=10)
                ax_polar2.tick_params(labelsize=10)
                
                # Plot time domain signal
                ax_time.plot(t, signal, label="Continuous Signal", color="b")
                ax_time.stem(sampled_t, sampled_signal, linefmt="r-", markerfmt="ro", basefmt="k")
                ax_time.set_xlabel("Time (s)", fontsize=9)
                ax_time.set_ylabel("Amplitude", fontsize=9)
                ax_time.set_title(f"Time Domain Signal (Window {self.current_window_index//N + 1})", fontsize=10)
                ax_time.legend(fontsize=8)
                ax_time.grid(True)
                ax_time.tick_params(labelsize=8)
                
                # Plot FFT spectrum
                ax_fft.plot(f_plot, np.abs(signal_fft), color="g")
                ax_fft.set_xlabel("Frequency (Hz)", fontsize=9)
                ax_fft.set_ylabel("Magnitude", fontsize=9)
                ax_fft.set_title("Frequency Spectrum", fontsize=10)
                ax_fft.grid(True)
                ax_fft.tick_params(labelsize=8)
                
                # Find maximum and minimum value indices
                max_idx = np.argmax(np.abs(signal_fft))  # Max value index
                min_idx = np.argmin(np.abs(signal_fft))  # Min value index
                
                # Get max and min values
                max_freq = f_plot[max_idx]
                max_value = np.abs(signal_fft[max_idx])
                min_freq = f_plot[min_idx]
                min_value = np.abs(signal_fft[min_idx])
                
                # Annotate max and min values on chart
                ax_fft.text(max_freq, max_value, f'{max_value:.2f}', fontsize=8, color='red', ha='left', va='bottom')
                ax_fft.text(min_freq, min_value, f'{min_value:.2f}', fontsize=8, color='blue', ha='left', va='top')
                
                # Plot red and blue dots at max and min positions
                ax_fft.plot(max_freq, max_value, 'ro', markersize=4)  # Red dot (max)
                ax_fft.plot(min_freq, min_value, 'bo', markersize=4)  # Blue dot (min)
                
                # Plot recursive phasor magnitude chart
                ax_mag.plot(np.abs(X_hat_v_original), label="Original", color="b", marker='*', markersize=4, linestyle='-', alpha=0.7)
                
                if self.enable_three_point_avg:
                    ax_mag.plot(np.abs(X_hat_v_filtered_for_display), label="Three-Point Avg", color="green", linewidth=2)
                    ax_mag.set_title(f"Phasor Magnitude (Window {self.current_window_index//N + 1})", fontsize=10)
                else:
                    ax_mag.set_title(f"Phasor Magnitude (Window {self.current_window_index//N + 1})", fontsize=10)
                
                ax_mag.set_xlabel("Sample Index", fontsize=9)
                ax_mag.set_ylabel("Magnitude", fontsize=9)
                ax_mag.legend(fontsize=8)
                ax_mag.grid(True)
                ax_mag.tick_params(labelsize=8)
                
                # Find max and min value indices (using currently displayed data)
                data_for_minmax = X_hat_v if self.enable_three_point_avg else X_hat_v_original
                max_idx = np.argmax(np.abs(data_for_minmax))  # Max value index
                min_idx = np.argmin(np.abs(data_for_minmax))  # Min value index
                
                # Get max and min values
                max_value = np.abs(data_for_minmax[max_idx])
                min_value = np.abs(data_for_minmax[min_idx])
                
                # Annotate max and min values on chart
                ax_mag.text(max_idx, max_value, f'{max_value:.2f}', fontsize=8, color='red', ha='left', va='bottom')
                ax_mag.text(min_idx, min_value, f'{min_value:.2f}', fontsize=8, color='blue', ha='left', va='top')
                
                # Plot red and blue dots at max and min positions
                ax_mag.plot(max_idx, max_value, 'ro', markersize=4)  # Red dot (max)
                ax_mag.plot(min_idx, min_value, 'bo', markersize=4)  # Blue dot (min)
                
                # Plot recursive phasor phase chart
                ax_phase.plot(np.angle(X_hat_v_original) * 180 / np.pi, label="Original", color="r", linewidth=1.5, alpha=0.7)
                
                if self.enable_three_point_avg:
                    ax_phase.plot(np.angle(X_hat_v_filtered_for_display) * 180 / np.pi, label="Three-Point Avg", color="darkgreen", linewidth=2)
                    ax_phase.set_title(f"Phasor Phase (Window {self.current_window_index//N + 1})", fontsize=10)
                else:
                    ax_phase.set_title(f"Phasor Phase (Window {self.current_window_index//N + 1})", fontsize=10)
                
                ax_phase.set_xlabel("Sample Index", fontsize=9)
                ax_phase.set_ylabel("Phase (degrees)", fontsize=9)
                ax_phase.legend(fontsize=8)
                ax_phase.grid(True)
                ax_phase.tick_params(labelsize=8)
                
                # Apply tight layout with fixed padding to prevent overlapping
                fig.tight_layout(pad=1.5)
                
                # Update phasor value display
                if self.recursive_phasors:
                    latest_recursive = self.recursive_phasors[-1]
                    self.recursive_re_output.value = f"Recursive Phasor Re: {latest_recursive.real:.4f}"
                    self.recursive_im_output.value = f"Recursive Phasor Im: {latest_recursive.imag:.4f}"
                    
                if self.non_recursive_phasors:
                    latest_non_recursive = self.non_recursive_phasors[-1]
                    self.non_recursive_re_output.value = f"Non-Recursive Phasor Re: {latest_non_recursive.real:.4f}"
                    self.non_recursive_im_output.value = f"Non-Recursive Phasor Im: {latest_non_recursive.imag:.4f}"
                
                # Update window index for next calculation
                self.current_window_index += N
                
                # Update status
                window_num = self.current_window_index // N
                if self.enable_three_point_avg:
                    self.status_output.value = f"<span style='color:green'>Window {window_num} - Three-Point Avg Enabled</span>"
                else:
                    self.status_output.value = f"<span style='color:blue'>Window {window_num} - All Charts Updated</span>"
                
                # Show figure - suppressing warnings
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    plt.show()
                    
        except ValueError:
            with self.output:
                clear_output(wait=True)
                print("Input Error: Please enter valid numerical values.")
                
        except Exception as e:
            with self.output:
                clear_output(wait=True)
                print(f"Error occurred: {str(e)}")


# Create a main function to run the application
def run_phasor_app():
    """Run the phasor analysis application"""
    app = WebPhasorApp()
    app.display()
    return app


# If run as main program, start the application
if __name__ == "__main__":
    run_phasor_app()